In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import spotter
from astropy.io import ascii

In [ ]:
#Get the data directory
cwd = os.getcwd()
data_dir = cwd.replace('Figure_7', 'Data\\')

data = ascii.read(data_dir + 'Final_Combined_Data.csv')
err_mask = (data['col3'] < 0.05) #Removing measurements with high uncertainty
data = data[err_mask]

time = data['col1']
flux = data['col2']
e_flux = data['col3']

In [ ]:
#Running the transit search via the spotter.py module

# main input: times, transit_flux, yerr, min_p, max_p, plot, stats, add_transit, transit_params
#transit_params input: planet radius (jupiter radii), t_inferior_conjunction, orbital period (days), stellar mass (kg), stellar radius (m), inclination (degrees), eccentricity, longitude of periastron (degrees)

transit_params = [2.0, 2.65, 10.0, 0.95*1.989*10**30, 0.96*6.96*10**8, 90, 0, 90] 
results = spotter.main(time, flux, e_flux, 3, 40, True, True, False, transit_params) 

In [ ]:
import matplotlib
plt.style.use('seaborn-dark-palette')
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 28}
matplotlib.rc('font', **font)

fig, axs = plt.subplots(2, 1)

plt.figure()
axs[0].axvline(results[2].period, alpha=0.4, lw=3, label='SDE Maximum', color='red')
for n in range(2, 10):
    axs[0].axvline(n*results[2].period, alpha=0.4, lw=1, linestyle="dashed")
    axs[0].axvline(results[2].period / n, alpha=0.4, lw=1, linestyle="dashed")
    if n == 9:
        axs[0].axvline(n*results[2].period, alpha=0.4, lw=1, linestyle="dashed", label='Resonances')
    
axs[0].set_ylabel(r'SDE')
axs[0].set_xlabel('Orbital Period (days)')
axs[0].plot(results[2].periods, results[2].power, color='black', lw=0.5)
axs[0].set_xlim(np.min(results[2].periods), np.max(results[2].periods))
axs[0].legend(loc = 'upper right', fontsize=18)
            
axs[1].plot(results[2].model_folded_phase, results[2].model_folded_model, color='red', label ='Model')
axs[1].errorbar(results[2].folded_phase, results[2].folded_y, fmt='.', color='blue', alpha=0.5, zorder=2, label='Period Folded Data') #Might error here
axs[1].set_xlim(0.45, 0.55)
axs[1].set_xlabel('Phase')
axs[1].set_ylabel('Normalised flux', fontsize = 25)
axs[1].legend(fontsize=18)

fig.set_size_inches(12, 8)
#fig.savefig('Ground_Based_search.pdf')            
plt.show()